##### Title: Monitoring how RTT varies based on the geolocation of the target
##### Author: Lotito Davide Pio
##### Date: 
<img src="./images/logoUnipvST.jpg" width="75" height="75">

## Some useful imports

In [8]:

from matplotlib import pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

ModuleNotFoundError: No module named 'geopandas'

## Read the file 

In [ ]:
df = pd.read_json("../sample.txt")
df

## Sort in ascending order by RTT

In [ ]:
df.sort_values('avgRTT')